In [ ]:
from simple_salesforce import Salesforce as sf
login_sf = sf(username = 'svc_rao@groupon.com', password = 'Password-1Je8IR8frn0gkbS68Eq4OHZmM', security_token = '')

import pandas as pd
from simple_salesforce import bulk
from megatron import saving as m
import pygsheets
import os


In [ ]:
gc = pygsheets.authorize(service_file=os.path.expanduser("~")+"\\.megatron\\google_service_account_secret.json")

sh = gc.open_by_key('1-XYkQmKeUKZRCuiLrzPuzqVYKttSbLW587U901rK734')

# define the following worksheets
live_accounts_1 = sh.worksheet_by_title("live_accounts_1")
live_accounts_2 = sh.worksheet_by_title("live_accounts_2")
live_accounts_3 = sh.worksheet_by_title("live_accounts_3")
db_output_1 = sh.worksheet_by_title("db_output_1")
db_output_2 = sh.worksheet_by_title("db_output_2")
db_output_3 = sh.worksheet_by_title("db_output_3")

live_acc_1 = live_accounts_1.get_values("B2","B10000")
live_acc_2 = live_accounts_2.get_values("B2","B10000")
live_acc_3 = live_accounts_3.get_values("B2","B10000")

acc_list_1 = []
acc_list_2 = []
acc_list_3 = []

print(len(live_acc_1))
print(len(live_acc_2))
print(len(live_acc_3))

for acc in live_acc_1:
    for x in acc:
        acc_list_1.append(x)

acc_list_query_1 = "','".join(acc_list_1)

for acc in live_acc_2:
    for x in acc:
        acc_list_2.append(x)

acc_list_query_2 = "','".join(acc_list_2)

for acc in live_acc_3:
    for x in acc:
        acc_list_3.append(x)

acc_list_query_3 = "','".join(acc_list_3)


In [ ]:
print(acc_list_query_3)


In [ ]:
### get DUNS numbers per account per possible in order to query the D&B Optimzer object
output = login_sf.bulk.Account.query(f"SELECT Id,DNBoptimizer__DnBCompanyRecord__c FROM Account WHERE Id in ('{acc_list_query_1}')")

df = pd.DataFrame(output)
# remove first column with attribute data
df = df.iloc[:,1:]
# filter out datapoints without DB_Ids numbers
db_map = df.dropna()

# prepeare list of DB_Ids to be queried
db_ids = db_map['DNBoptimizer__DnBCompanyRecord__c'].tolist()
db_id_list = "','".join(db_ids)

### get D&B Optimizer data for the DUNS numbers found
output = login_sf.bulk.DNBoptimizer__DnBCompanyRecord__c.query(f"SELECT Id,DNBoptimizer__BusinessName__c,DNBoptimizer__DUNSNumber__c,DNBoptimizer__GlobalUltimateDUNSNumber__c,DNBoptimizer__DelistedIndicator__c,DNBoptimizer__EmployeeCountTotal__c,DNBoptimizer__LocationType__c,DNBoptimizer__NAICSDescription1__c,DNBoptimizer__NAICSDescription2__c,DNBoptimizer__NAICSDescription3__c,DNBoptimizer__OutofBusinessIndicator__c,DNBoptimizer__PrimaryCityName__c,DNBoptimizer__PrimaryCountryCode_ISO__c,DNBoptimizer__PrimaryCountryName__c,DNBoptimizer__PrimaryPostalCode__c,DNBoptimizer__PrimaryStreetAddress__c,DNBoptimizer__TelephoneNumber__c,DNBoptimizer__WebAddress__c,DNBoptimizer__YearStarted__c FROM DNBoptimizer__DnBCompanyRecord__c where Id in ('{db_id_list}')")

#prepare table from sf_output
df = pd.DataFrame(output)
db_data = df.iloc[:,1:]

### Join datasets
# rename columns in order to join
db_map.columns=['AccountId','Id']

df = db_map.join(db_data.set_index('Id'), on='Id')

### push data to google-sheet
m.df_to_gdoc(df,'1wHvQnt1eIs0naWO2fdLglCY0NQjON_4OlQzdGukZj-8','db_output_1')

print("Success")


In [ ]:
### get DUNS numbers per account per possible in order to query the D&B Optimzer object
output = login_sf.bulk.Account.query(f"SELECT Id,DNBoptimizer__DnBCompanyRecord__c FROM Account WHERE Id in ('{acc_list_query_2}')")

df = pd.DataFrame(output)
# remove first column with attribute data
df = df.iloc[:,1:]
# filter out datapoints without DB_Ids numbers
db_map = df.dropna()

# prepeare list of DB_Ids to be queried
db_ids = db_map['DNBoptimizer__DnBCompanyRecord__c'].tolist()
db_id_list = "','".join(db_ids)

### get D&B Optimizer data for the DUNS numbers found
output = login_sf.bulk.DNBoptimizer__DnBCompanyRecord__c.query(f"SELECT Id,DNBoptimizer__BusinessName__c,DNBoptimizer__DUNSNumber__c,DNBoptimizer__GlobalUltimateDUNSNumber__c,DNBoptimizer__DelistedIndicator__c,DNBoptimizer__EmployeeCountTotal__c,DNBoptimizer__LocationType__c,DNBoptimizer__NAICSDescription1__c,DNBoptimizer__NAICSDescription2__c,DNBoptimizer__NAICSDescription3__c,DNBoptimizer__OutofBusinessIndicator__c,DNBoptimizer__PrimaryCityName__c,DNBoptimizer__PrimaryCountryCode_ISO__c,DNBoptimizer__PrimaryCountryName__c,DNBoptimizer__PrimaryPostalCode__c,DNBoptimizer__PrimaryStreetAddress__c,DNBoptimizer__TelephoneNumber__c,DNBoptimizer__WebAddress__c,DNBoptimizer__YearStarted__c FROM DNBoptimizer__DnBCompanyRecord__c where Id in ('{db_id_list}')")

#prepare table from sf_output
df = pd.DataFrame(output)
db_data = df.iloc[:,1:]

### Join datasets
# rename columns in order to join
db_map.columns=['AccountId','Id']

df = db_map.join(db_data.set_index('Id'), on='Id')

### push data to google-sheet
m.df_to_gdoc(df,'1wHvQnt1eIs0naWO2fdLglCY0NQjON_4OlQzdGukZj-8','db_output_2')

print("Success")


In [ ]:
### get DUNS numbers per account per possible in order to query the D&B Optimzer object
output = login_sf.bulk.Account.query(f"SELECT Id,DNBoptimizer__DnBCompanyRecord__c FROM Account WHERE Id in ('{acc_list_query_3}')")

df = pd.DataFrame(output)
# remove first column with attribute data
df = df.iloc[:,1:]
# filter out datapoints without DB_Ids numbers
db_map = df.dropna()

# prepeare list of DB_Ids to be queried
db_ids = db_map['DNBoptimizer__DnBCompanyRecord__c'].tolist()
db_id_list = "','".join(db_ids)

### get D&B Optimizer data for the DUNS numbers found
output = login_sf.bulk.DNBoptimizer__DnBCompanyRecord__c.query(f"SELECT Id,DNBoptimizer__BusinessName__c,DNBoptimizer__DUNSNumber__c,DNBoptimizer__GlobalUltimateDUNSNumber__c,DNBoptimizer__DelistedIndicator__c,DNBoptimizer__EmployeeCountTotal__c,DNBoptimizer__LocationType__c,DNBoptimizer__NAICSDescription1__c,DNBoptimizer__NAICSDescription2__c,DNBoptimizer__NAICSDescription3__c,DNBoptimizer__OutofBusinessIndicator__c,DNBoptimizer__PrimaryCityName__c,DNBoptimizer__PrimaryCountryCode_ISO__c,DNBoptimizer__PrimaryCountryName__c,DNBoptimizer__PrimaryPostalCode__c,DNBoptimizer__PrimaryStreetAddress__c,DNBoptimizer__TelephoneNumber__c,DNBoptimizer__WebAddress__c,DNBoptimizer__YearStarted__c FROM DNBoptimizer__DnBCompanyRecord__c where Id in ('{db_id_list}')")

#prepare table from sf_output
df = pd.DataFrame(output)
db_data = df.iloc[:,1:]

### Join datasets
# rename columns in order to join
db_map.columns=['AccountId','Id']

df = db_map.join(db_data.set_index('Id'), on='Id')

### push data to google-sheet
m.df_to_gdoc(df,'1wHvQnt1eIs0naWO2fdLglCY0NQjON_4OlQzdGukZj-8','db_output_3')

print("Success")
